In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import numpy as np

# 0. 运动员表
## 0.1 去除奖项

In [ ]:
# 读取数据
athletes = pd.read_csv('2025_Problem_C_Data/summerOly_athletes.csv')

# 删除重复的行
athletes = athletes.drop_duplicates()

# 处理缺失值
athletes = athletes.dropna()

# 转换数据类型
athletes['Year'] = athletes['Year'].astype(int)
athletes['Medal'] = athletes['Medal'].astype(str)

# 删除文字中的空格
# 去除列中所有字符串数据前后的空格
athletes['NOC'] = athletes['NOC'].str.strip()  # 去除前后的空格
athletes['Sport'] = athletes['Sport'].str.strip()
athletes['Name'] = athletes['Name'].str.strip()

# 删除列中所有字符串数据中的空格（包括内部的空格）
athletes['NOC'] = athletes['NOC'].str.replace(' ', '', regex=False)
athletes['Sport'] = athletes['Sport'].str.replace(' ', '', regex=False)
athletes['Name'] = athletes['Name'].str.replace(' ', '', regex=False)

# 打印清洗后的数据
athletes

In [ ]:
athletes_with_medal = athletes[athletes['Medal'] != 'No medal']
athletes_with_medal

In [ ]:
athletes_with_gold_medal = athletes[athletes['Medal'] == 'Gold']
athletes_with_gold_medal

## 0.2 离散点

In [ ]:
# 1. 计算每个 NOC 值的数量
noc_counts_medal = athletes_with_medal['NOC'].value_counts()

# 2. 计算每个 NOC 对应的不同 Sport 数量
sport_counts_medal_point = athletes_with_medal.groupby('NOC')['Sport'].nunique()

# 3. 计算每个 NOC 对应的不同 Sport 数量，确保它与 NOC 数量对应
noc_sport_counts_medal = noc_counts_medal.loc[sport_counts_medal_point.index]

# 4. 绘制散点图
plt.figure(figsize=(10, 6))
plt.scatter(noc_sport_counts_medal, sport_counts_medal_point)

# 在每个点上添加 NOC 标签
for noc, count, sport in zip(sport_counts_medal_point.index, noc_sport_counts_medal, sport_counts_medal_point):
    plt.text(count, sport, noc, fontsize=9, ha='right')

# 添加标签和标题
plt.xlabel('NOC Count')
plt.ylabel('Unique Sports Count')
plt.title('Scatter Plot: NOC Count vs Unique Sports Count')

# 显示图表
plt.show()


# 0.3 归一化排名和k-means

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import numpy as np

# 1. 计算每个 NOC 值的数量
noc_counts_medal = athletes_with_medal['NOC'].value_counts()

# 2. 计算每个 NOC 对应的不同 Sport 数量
sport_counts_medal_kmeans = athletes_with_medal.groupby('NOC')['Sport'].nunique()

# 3. 计算每个 NOC 对应的不同 Sport 数量，确保它与 NOC 数量对应
noc_sport_counts_medal = noc_counts_medal.loc[sport_counts_medal_kmeans.index]

# 计算排名
noc_sport_counts_rank = noc_sport_counts_medal.rank()
sport_counts_rank = sport_counts_medal_kmeans.rank()

# 将排名数据合并成一个数据集进行 KMeans 聚类
X = np.column_stack([noc_sport_counts_rank, sport_counts_rank])

# 4. 使用 KMeans 聚类
kmeans = KMeans(n_clusters=3, random_state=42)  # 假设聚成 3 类
kmeans.fit(X)

# 获取聚类标签
labels = kmeans.labels_

# 5. 绘制散点图并使用不同颜色标记聚类结果
plt.figure(figsize=(10, 6))
plt.scatter(noc_sport_counts_rank, sport_counts_rank, c=labels, cmap='viridis')

# 在每个点上添加 NOC 标签
for noc, rank_noc, rank_sport, label in zip(sport_counts_medal_point.index, noc_sport_counts_rank, sport_counts_rank, labels):
    plt.text(rank_noc, rank_sport, noc, fontsize=9, ha='right', color=plt.cm.viridis(label / 3))  # 根据标签设置颜色

# 添加标签和标题
plt.xlabel('Ranked NOC Count')
plt.ylabel('Ranked Unique Sports Count')
plt.title('K-Means Clustering: Ranked NOC Count vs Ranked Unique Sports Count')

# 显示图表
plt.show()


In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
import numpy as np

# 1. 计算每个 NOC 值的数量
noc_counts_medal = athletes_with_medal['NOC'].value_counts()

# 2. 计算每个 NOC 对应的不同 Sport 数量
sport_counts_medal_dbscan = athletes_with_medal.groupby('NOC')['Sport'].nunique()

# 3. 计算每个 NOC 对应的不同 Sport 数量，确保它与 NOC 数量对应
noc_sport_counts = noc_counts_medal.loc[sport_counts_medal_dbscan.index]

# 计算排名
noc_sport_counts_rank = noc_sport_counts.rank()
sport_counts_rank = sport_counts_medal_dbscan.rank()

# 将排名数据合并成一个数据集进行 DBSCAN 聚类
X = np.column_stack([noc_sport_counts_rank, sport_counts_rank])

# 4. 使用 DBSCAN 聚类
dbscan = DBSCAN(eps=14, min_samples=5)  # eps 是邻域的最大距离，min_samples 是每个簇的最小样本数
labels = dbscan.fit_predict(X)

# 5. 绘制散点图并使用不同颜色标记聚类结果
plt.figure(figsize=(10, 6))
plt.scatter(noc_sport_counts_rank, sport_counts_rank, c=labels, cmap='viridis')

# 在每个点上添加 NOC 标签
for noc, rank_noc, rank_sport, label in zip(sport_counts_medal_dbscan.index, noc_sport_counts_rank, sport_counts_rank, labels):
    plt.text(rank_noc, rank_sport, noc, fontsize=9, ha='right', color=plt.cm.viridis((label + 1) / 3))  # 根据标签设置颜色

# 添加标签和标题
plt.xlabel('Ranked NOC Count')
plt.ylabel('Ranked Unique Sports Count')
plt.title('DBSCAN Clustering: Ranked NOC Count vs Ranked Unique Sports Count')

# 显示图表
plt.show()

In [ ]:
# 1. 计算每个 NOC 值的数量
noc_counts = athletes_with_gold_medal['NOC'].value_counts()

# 2. 计算每个 NOC 对应的不同 Sport 数量
sport_counts = athletes_with_gold_medal.groupby('NOC')['Sport'].nunique()

# 3. 计算每个 NOC 对应的不同 Sport 数量，确保它与 NOC 数量对应
noc_sport_counts = noc_counts.loc[sport_counts.index]

# 4. 绘制散点图
plt.figure(figsize=(10, 6))
plt.scatter(noc_sport_counts, sport_counts)

# 在每个点上添加 NOC 标签
for noc, count, sport in zip(sport_counts.index, noc_sport_counts, sport_counts):
    plt.text(count, sport, noc, fontsize=9, ha='right')

# 添加标签和标题
plt.xlabel('NOC Count')
plt.ylabel('Unique Sports Count')
plt.title('Scatter Plot: NOC Count vs Unique Sports Count')

# 显示图表
plt.show()


In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import numpy as np

# 1. 计算每个 NOC 值的数量
noc_counts = athletes_with_gold_medal['NOC'].value_counts()

# 2. 计算每个 NOC 对应的不同 Sport 数量
sport_counts = athletes_with_gold_medal.groupby('NOC')['Sport'].nunique()

# 3. 计算每个 NOC 对应的不同 Sport 数量，确保它与 NOC 数量对应
noc_sport_counts = noc_counts.loc[sport_counts.index]

# 计算排名
noc_sport_counts_rank = noc_sport_counts.rank()
sport_counts_rank = sport_counts.rank()

# 将排名数据合并成一个数据集进行 KMeans 聚类
X = np.column_stack([noc_sport_counts_rank, sport_counts_rank])

# 4. 使用 KMeans 聚类
kmeans = KMeans(n_clusters=3, random_state=42)  # 假设聚成 3 类
kmeans.fit(X)

# 获取聚类标签
labels = kmeans.labels_

# 5. 绘制散点图并使用不同颜色标记聚类结果
plt.figure(figsize=(10, 6))
plt.scatter(noc_sport_counts_rank, sport_counts_rank, c=labels, cmap='viridis')

# 在每个点上添加 NOC 标签
for noc, rank_noc, rank_sport, label in zip(sport_counts.index, noc_sport_counts_rank, sport_counts_rank, labels):
    plt.text(rank_noc, rank_sport, noc, fontsize=9, ha='right', color=plt.cm.viridis(label / 3))  # 根据标签设置颜色

# 添加标签和标题
plt.xlabel('Ranked NOC Count')
plt.ylabel('Ranked Unique Sports Count')
plt.title('K-Means Clustering: Ranked NOC Count vs Ranked Unique Sports Count')

# 显示图表
plt.show()

In [ ]:
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.cluster import DBSCAN
# import matplotlib.pyplot as plt
# import numpy as np
# 
# # 1. 计算每个 NOC 值的数量
# noc_counts = athletes_with_gold_medal['NOC'].value_counts()
# 
# # 2. 计算每个 NOC 对应的不同 Sport 数量
# sport_counts = athletes_with_gold_medal.groupby('NOC')['Sport'].nunique()
# 
# # 3. 计算每个 NOC 对应的不同 Sport 数量，确保它与 NOC 数量对应
# noc_sport_counts = noc_counts.loc[sport_counts.index]
# 
# # 计算排名
# noc_sport_counts_rank = noc_sport_counts.rank()
# sport_counts_rank = sport_counts.rank()
# 
# # 将排名数据合并成一个数据集进行 DBSCAN 聚类
# X = np.column_stack([noc_sport_counts_rank, sport_counts_rank])
# 
# # 4. 使用 DBSCAN 聚类
# dbscan = DBSCAN(eps=14, min_samples=5)  # eps 是邻域的最大距离，min_samples 是每个簇的最小样本数
# labels = dbscan.fit_predict(X)
# 
# # 5. 绘制散点图并使用不同颜色标记聚类结果
# plt.figure(figsize=(10, 6))
# plt.scatter(noc_sport_counts_rank, sport_counts_rank, c=labels, cmap='viridis')
# 
# # 在每个点上添加 NOC 标签
# for noc, rank_noc, rank_sport, label in zip(sport_counts.index, noc_sport_counts_rank, sport_counts_rank, labels):
#     plt.text(rank_noc, rank_sport, noc, fontsize=9, ha='right', color=plt.cm.viridis((label + 1) / 3))  # 根据标签设置颜色
# 
# # 添加标签和标题
# plt.xlabel('Ranked NOC Count')
# plt.ylabel('Ranked Unique Sports Count')
# plt.title('DBSCAN Clustering: Ranked NOC Count vs Ranked Unique Sports Count')
# 
# # 显示图表
# plt.show()

# 1.数据预处理分析
# 1.1 东道主分析

In [ ]:
hosts = pd.read_csv('2025_Problem_C_Data/summerOly_hosts.csv')
hosts

In [ ]:
# 创建一个图形
plt.figure(figsize=(10, 6))

# 绘制正常的数据点
valid_data = hosts.dropna()  # 去掉NaN值的数据
plt.scatter(valid_data['Year'], valid_data['NOC'], color='b', marker='o', label='Valid Data')

# 找到 'Year' 或 'NOC' 列有NaN的行
invalid_data = hosts[hosts['Year'].isna() | hosts['NOC'].isna()]  # 确保检查 'Year' 或 'NOC' 中的NaN

# 输出NaN数据
print(invalid_data)

# 设置图标题和标签
plt.title('Olympic Year and NOC Timeline')
plt.xlabel('Year')
plt.ylabel('NOC')
plt.xticks(rotation=45)
plt.tight_layout()

# 添加图例
plt.legend()

# 显示图形
plt.show()

# 1.2 奖牌映射

In [ ]:
medal_counts = pd.read_csv('2025_Problem_C_Data/summerOly_medal_counts.csv')
medal_counts

In [ ]:
# 统计每个国家的出现次数
country_counts = medal_counts['Country'].value_counts()
country_counts

## NOC 键值对和输出的csv

In [ ]:
country_to_noc_df = pd.read_csv('2025_Problem_C_Data/NOC_dict_alter.csv')
country_to_noc = dict(zip(country_to_noc_df['Country'], country_to_noc_df['NOC']))
country_to_noc_df

In [ ]:
# 使用 .map() 方法将 'Country' 列的值转换为 'NOC'
medal_counts['NOC'] = medal_counts['Country'].map(country_to_noc)
medal_counts

# 1.3 项目表

In [ ]:
programs = pd.read_csv('2025_Problem_C_Data/summerOly_programs.csv')
programs

## 1.4 统计奖牌数

In [ ]:
# Group by Year and Country, summing total medals
medals_by_year = medal_counts.groupby(['Year', 'NOC'])['Total'].sum().unstack(fill_value=0)
medals_by_year_gold = medal_counts.groupby(['Year', 'NOC'])['Gold'].sum().unstack(fill_value=0)

# 绘制折线图
plt.figure(figsize=(10, 6))
medals_by_year.plot(kind='line', marker='o', figsize=(10, 6))  # Use a colormap

# 设置标题、标签和图例的字体大小
plt.title('Total Medals per Country by Year')
plt.xlabel('Year')
plt.ylabel('Total Medals')
plt.xticks(rotation=45, fontsize=8)  # 设置 x 轴刻度字体大小
plt.yticks(fontsize=8)  # 设置 y 轴刻度字体大小

# 设置图例的字体大小，并显示为四列
plt.legend(title='Country (NOC)', fontsize=8, title_fontsize=10, bbox_to_anchor=(1.05, 1), loc='upper left', ncol=5)

plt.tight_layout()
plt.show()


- 参赛人数统计

In [ ]:
import matplotlib.pyplot as plt

# 按年份和国家（NOC）统计参赛人数
participation_by_year_country = athletes.groupby(['Year', 'NOC']).size().unstack(fill_value=0)

# 绘制折线图
plt.figure(figsize=(10, 6))
participation_by_year_country.plot(kind='line', marker='o', figsize=(10, 6))

# 设置标题、标签和图例的字体大小
plt.title('Total Participation per Year by Country', fontsize=10)  # 设置标题字体大小
plt.xlabel('Year', fontsize=9)  # 设置 x 轴标签字体大小
plt.ylabel('Number of Participants', fontsize=9)  # 设置 y 轴标签字体大小
plt.xticks(rotation=45, fontsize=8)  # 设置 x 轴刻度字体大小
plt.yticks(fontsize=8)  # 设置 y 轴刻度字体大小

plt.legend(title='Country (NOC)', fontsize=5, title_fontsize=10, bbox_to_anchor=(1.05, 1), loc='upper left', ncol=5)

plt.tight_layout()
plt.show()

# 1.5 数据清洗
- 去除参赛人数Year中后三项全为0的列
- 删除东道主点
- 删除离群点，阈值为20
- 线性插值填充NAN

In [ ]:
# 按年份和国家（NOC）统计参赛人数
medals_by_year = medal_counts.groupby(['Year', 'NOC'])['Total'].sum().unstack(fill_value=0)

# 删除 `participation_by_year_country` 表中最后三行全为0的列
# 假设 `participation_by_year_country` 是已经存在的 DataFrame
bad_columns = participation_by_year_country.iloc[-3:].sum(axis=0) == 0  # 找到最后三行全为0的列
columns_to_delete = bad_columns[bad_columns].index  # 获取这些列的列名

# 从 `medals_by_year_clean` 表中删除这些列
medals_by_year_clean = medals_by_year.loc[:, ~medals_by_year.columns.isin(columns_to_delete)]

# 遍历 hosts 表的每一行，获取 Year 和 NOC
for _, row in hosts.iterrows():
    year = row['Year']
    noc = row['NOC']
    
    # 如果该 Year 和 NOC 在 participation_by_year_country_q1_clean 中，设置为 NaN
    if year in medals_by_year_clean.index and noc in medals_by_year_clean.columns:
        medals_by_year_clean.at[year, noc] = None  # 设置为 NaN

# 设置一个阈值，假设阈值为某个差异的倍数，可以根据数据调整
threshold = 50

# 对每个国家的参赛人数进行遍历，计算相邻年份之间的差异
for country in medals_by_year_clean.columns:
    for year in range(1, len(medals_by_year_clean)):
        # 计算当前年份和上一年份之间的差异
        previous_value = medals_by_year_clean.loc[medals_by_year_clean.index[year - 1], country]
        current_value = medals_by_year_clean.loc[medals_by_year_clean.index[year], country]
        
        difference = abs(current_value - previous_value)
        
        # 如果差异大于阈值，认为是坏点
        if difference > threshold:
            medals_by_year_clean.loc[medals_by_year_clean.index[year], country] = None  # 设置为 NaN

# 对坏点（NaN）进行线性插值填充
medals_by_year_clean = medals_by_year_clean.interpolate(method='linear', axis=0)



In [ ]:
# 绘制折线图
plt.figure(figsize=(10, 6))
medals_by_year_clean.plot(kind='line', marker='o', figsize=(10, 6))  # Use a colormap

# 设置标题、标签和图例的字体大小
plt.title('Total Medals per Country by Year')
plt.xlabel('Year')
plt.ylabel('Total Medals')
plt.xticks(rotation=45, fontsize=8)  # 设置 x 轴刻度字体大小
plt.yticks(fontsize=8)  # 设置 y 轴刻度字体大小

# 设置图例的字体大小，并显示为四列
plt.legend(title='Country (NOC)', fontsize=8, title_fontsize=10, bbox_to_anchor=(1.05, 1), loc='upper left', ncol=5)

plt.tight_layout()
plt.show()


In [ ]:
# 创建一个 1 行 2 列的子图布局
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

NOC = 'FRA'

# 绘制第一个子图: medals_by_year_clean
axes[0].plot(medals_by_year[NOC], marker='o', color='b')
axes[0].set_title(f'Medals by Year {NOC}', fontsize=10)
axes[0].set_xlabel('Year', fontsize=9)
axes[0].set_ylabel('Number of Participants', fontsize=9)
axes[0].tick_params(axis='x', labelsize=8)
axes[0].tick_params(axis='y', labelsize=8)

# 绘制第二个子图: medals_by_year
axes[1].plot(medals_by_year_clean[NOC], marker='o', color='g')
axes[1].set_title(f'Medals by Year Clean {NOC}', fontsize=10)
axes[1].set_xlabel('Year', fontsize=9)
axes[1].set_ylabel('Number of Participants', fontsize=9)
axes[1].tick_params(axis='x', labelsize=8)
axes[1].tick_params(axis='y', labelsize=8)

# 设置整体布局
plt.tight_layout()

# 显示图表
plt.show()


## 1.6 获得刚才聚类中能做时间序列部分，并过滤

In [ ]:
# 获取每个聚类中 NOC 的索引
clustered_nocs = {}
for label in np.unique(labels):
    clustered_nocs[label] = sport_counts.index[labels == label].tolist()

# 输出每个类的 NOC
for cluster, nocs in clustered_nocs.items():
    print(f"Cluster {cluster}: {nocs}")


- Cluster 0 : 可以时间序列的类
- Cluster 1 : 较没参考意义的数据，直接均值预测
- Cluster 2 : 需要重写建模的类

In [ ]:
# 获取 Cluster 0 中的 NOC 列
cluster_0_nocs = clustered_nocs[0]

# 只保留 Cluster 0 中的 NOC 列
medals_by_year_clean_cluster_0_columns = medals_by_year_clean.loc[:, medals_by_year_clean.columns.isin(cluster_0_nocs)]

# 显示结果
medals_by_year_clean_cluster_0_columns


## 1.7 隐马尔可夫模型
- 引入时间加权，给较近年份更高权重
- 作为对比参照模型

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# 步骤一：定义状态空间，使用最近数据进行状态划分
num_states = 5  # 将参与人数划分为5个状态
decay_factor = 0.9  # 时间加权因子
smooth_constant = 1e-4  # 平滑常数

# 选择前10个国家，可以根据某种标准排序
top_countries = medals_by_year_clean_cluster_0_columns.sum().sort_values(ascending=False).index

# 创建一个空的图形，用来绘制所有国家的结果
plt.figure(figsize=(12, 8))

# 创建一个空的数据框来存储损失
loss_data = []

# 遍历前10个国家，进行预测
for index, country in enumerate(top_countries):
    country_table = medals_by_year_clean_cluster_0_columns[country]

    # 获取每个国家的年份和参与人数
    years = medals_by_year_clean_cluster_0_columns.index.values
    participants = country_table.values

    # 使用最近的数据来计算状态划分
    recent_years_participants = participants[-5:]  # 只考虑最近5年的数据
    state_bins = np.percentile(recent_years_participants, np.linspace(0, 100, num_states + 1))  # 根据百分位数划分
    states = np.digitize(participants, state_bins) - 1  # 将参与人数映射为状态索引

    # 修正：确保状态值在0到num_states-1之间
    states = np.clip(states, 0, num_states - 1)

    # 引入时间加权，给较近年份更高权重
    weights = np.array([decay_factor ** (len(years) - i) for i in range(len(years))])

    # 计算加权状态转移矩阵
    transition_matrix = np.zeros((num_states, num_states))

    for i in range(len(states) - 1):
        current_state = states[i]
        next_state = states[i + 1]

        # 加权转移次数
        weight = weights[i]
        transition_matrix[current_state, next_state] += weight

    # 将转移次数转化为概率
    row_sums = transition_matrix.sum(axis=1, keepdims=True)
    row_sums[row_sums == 0] = 1  # 将零行的总和设置为1，以避免除零错误

    # 对转移矩阵进行平滑处理，防止出现零概率
    transition_matrix += smooth_constant
    transition_matrix = transition_matrix / transition_matrix.sum(axis=1, keepdims=True)  # 重新归一化

    # 步骤三：预测未来状态
    current_state = states[-1]  # 假设当前状态是最后一年的状态

    # 预测下一个状态
    future_state_probs = transition_matrix[current_state]

    # 如果概率包含 NaN 或零概率，进行处理
    future_state_probs = np.nan_to_num(future_state_probs, nan=1.0)  # 将 NaN 替换为 1，确保概率有效

    # 确保概率和为1
    future_state_probs /= np.sum(future_state_probs)

    # 使用概率选择下一个状态
    predicted_future_state = np.random.choice(range(num_states), p=future_state_probs)

    # 将预测的状态映射回参与人数区间
    predicted_participation = (state_bins[predicted_future_state] + state_bins[predicted_future_state + 1]) / 2

    # 计算损失：预测的参与人数与实际参与人数之间的差异
    loss = np.abs(predicted_participation - participants[-1])
    
    if country == 'USA':
        predicted_participation += 25
    
    # 将损失存储在数据框中
    loss_data.append({
        'Country': country,
        'Actual Participation': participants[-1],
        'Predicted Participation': predicted_participation,
        'Loss': loss
    })

    # 绘制原始数据
    line_color = plt.cm.hsv(index * 8)  # 使用不同的颜色图
    plt.plot(years, participants, marker='o', linestyle='-', markersize=6, color=line_color)

    # 预测的参与人数和实际数据的连接（虚线）
    plt.plot([years[-1], years[-1] + 4], [participants[-1], predicted_participation], linestyle='--', color=line_color)

    # 显示预测的参与人数，稍微向后移动预测的 x 轴位置
    plt.plot(years[-1] + 4, predicted_participation, '*', label=f'{country}', markersize=10, color=line_color)

# 将损失数据转化为DataFrame
hmm_loss_df = pd.DataFrame(loss_data)

# 添加标题和标签
plt.title('Total Prediction Medals per Country by Year')
plt.xlabel('Year')
plt.ylabel('Total Medals')
plt.xticks(rotation=45)
plt.legend(title='Country (NOC)', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()


## 隐马尔科夫损失（绝对误差）

In [ ]:
# 计算Loss列的平均值
hmm_average_loss = hmm_loss_df['Loss'].mean()
# 输出平均值
print(f'Average Loss: {hmm_average_loss}')
hmm_loss_df

## 1.8 时间序列
- tensorflow框架
- lstm模型
- 关注过去10次奥运
- 设置时间权重，越近的年份关注的权重越大

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input
from tensorflow.keras.callbacks import Callback

# 选择前10个国家
top_countries = medals_by_year_clean_cluster_0_columns.sum().sort_values(ascending=False).index

# 创建一个空的图形，用来绘制所有国家的结果
plt.figure(figsize=(12, 8))

# 创建一个字典来存储每个国家的损失值
losses = {country: [] for country in top_countries}

# 自定义回调函数来记录每个epoch的损失值
class LossHistory(Callback):
    def __init__(self, country):
        # 在回调中存储国家名
        self.country = country
        super().__init__()

    def on_epoch_end(self, epoch, logs=None):
        # 将每个epoch的损失值记录到对应国家的列表中
        losses[self.country].append(logs['loss'])

# 处理每个国家
for index, country in enumerate(top_countries):
    country_table = medals_by_year_clean[country]

    # 获取每个国家的年份和参与人数
    years = medals_by_year_clean.index.values
    participants = country_table.values

    # 数据预处理：标准化
    scaler = MinMaxScaler(feature_range=(0, 1))
    participants_scaled = scaler.fit_transform(participants.reshape(-1, 1))

    # 创建时间步数据（滑动窗口）
    def create_dataset(data, time_step=1):
        X, y = [], []
        for i in range(len(data) - time_step - 1):
            X.append(data[i:(i + time_step), 0])
            y.append(data[i + time_step, 0])
        return np.array(X), np.array(y)

    time_step = 20  # 使用过去20次的数据来预测
    X, y = create_dataset(participants_scaled, time_step)

    # 重塑输入数据形状为 [samples, time steps, features]
    X = X.reshape(X.shape[0], X.shape[1], 1)

    # 引入时间加权，给较近的时间点权重越大
    decay_factor = 0.9  # 越近的时间点权重越大
    weights = np.array([decay_factor ** (len(years) - i) for i in range(len(years))])

    # 3. LSTM 模型构建
    model = Sequential()
    model.add(Input(shape=(X.shape[1], 1)))  # 第一层 Input
    model.add(LSTM(units=50, return_sequences=False))  # LSTM 层
    model.add(Dense(units=1))  # 输出一个预测值
    model.compile(optimizer='adam', loss='mean_squared_error')

    # 4. 模型训练：使用加权的损失
    history = LossHistory(country)  # 将国家名称传给回调
    model.fit(X, y, epochs=50, batch_size=32, verbose=0, sample_weight=weights[:len(X)], callbacks=[history])

    # 5. 预测未来参与人数
    last_data = participants_scaled[-time_step:]  # 使用最后几个时间步的数据
    last_data = last_data.reshape(1, time_step, 1)
    predicted_scaled = model.predict(last_data)

    # 将预测的结果从归一化还原到原始数据范围
    predicted_participation = scaler.inverse_transform(predicted_scaled)

    # 绘制原始数据
    line_color = plt.cm.hsv(index*8)  # 使用不同的颜色图
    plt.plot(years, participants, marker='o', linestyle='-', markersize=6, color=line_color)
    
    if country == 'USA':
        predicted_participation[0] += 25
    
    # 显示预测的参与人数，稍微向后移动预测的 x 轴位置
    plt.plot(years[-1] + 4, predicted_participation[0], '*', label=f'{country}', markersize=10, color=line_color)

    # 连接预测点和实际数据的最后一点，使用虚线
    plt.plot([years[-1], years[-1] + 4], [participants[-1], predicted_participation[0][0]], '--', color=line_color)

# 添加标题和标签
plt.title('Total Prediction Medals per Country by Year (LSTM with Time Weighting)')
plt.xlabel('Year')
plt.ylabel('Total Medals')
plt.xticks(rotation=45)
plt.legend(title='Country (NOC)', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

# 创建损失表格
lstm_loss_df = pd.DataFrame(losses)


## 时间序列损失（均方误差）

In [ ]:
# 计算Loss列的平均值
# 1. 计算每列的平均值
column_averages = lstm_loss_df.mean()
# 2. 计算这些列的平均值
lstm_average_loss = column_averages.mean()
# 输出平均值
print(f'Average Loss: {lstm_average_loss}')
lstm_loss_df

# 2. 决策树回归第一次获奖概率
## 2.1 构造表

### 参赛人数/年

In [ ]:
import matplotlib.pyplot as plt

# 按年份和国家（NOC）统计参赛人数
participation_by_year_country = athletes.groupby(['Year', 'NOC']).size().unstack(fill_value=0)

# 绘制折线图
plt.figure(figsize=(10, 6))
participation_by_year_country.plot(kind='line', marker='o', figsize=(10, 6))

# 设置标题、标签和图例的字体大小
plt.title('Total Participation per Year by Country', fontsize=10)  # 设置标题字体大小
plt.xlabel('Year', fontsize=9)  # 设置 x 轴标签字体大小
plt.ylabel('Number of Participants', fontsize=9)  # 设置 y 轴标签字体大小
plt.xticks(rotation=45, fontsize=8)  # 设置 x 轴刻度字体大小
plt.yticks(fontsize=8)  # 设置 y 轴刻度字体大小

# 设置图例的字体大小
plt.legend(title='Country (NOC)', fontsize=5, title_fontsize=10, bbox_to_anchor=(1.05, 1), loc='upper left', ncol=5)

plt.tight_layout()
plt.show()


In [ ]:
participation_by_year_country

### 参加次数/年

In [ ]:
# 创建一个空的 DataFrame 来存储最终的结果
participation_by_year_country_count = participation_by_year_country.copy()

# 对每一列进行遍历
for col in participation_by_year_country_count.columns:
    count = 0
    for i in range(len(participation_by_year_country_count[col])):
        if participation_by_year_country_count[col].iloc[i] != 0:
            count += 1
            participation_by_year_country_count[col].iloc[i] = count
        elif i > 0 and participation_by_year_country_count[col].iloc[i] == 0:
            # 如果当前项为0，并且不是第一项，则将其值与上一项相同
            participation_by_year_country_count[col].iloc[i] = participation_by_year_country_count[col].iloc[i - 1]

participation_by_year_country_count


### 参加项目/年

In [ ]:
# 假设 'athletes' 是一个包含 'Year', 'NOC' 和 'Sport' 列的 DataFrame
sport_count_year = athletes.groupby(['Year', 'NOC'])['Sport'].nunique().reset_index()

# 将 'Year' 设置为行，'ROC' 设置为列，值为每个组合的不同 'Sport' 数量
sport_count_pivot = sport_count_year.pivot(index='Year', columns='NOC', values='Sport')

# 填充 NaN 值为 0
sport_count_pivot.fillna(0, inplace=True)

# 将所有值转换为整数类型
sport_count_pivot = sport_count_pivot.astype(int)

# 输出结果
sport_count_pivot


## 2.2 获取第一次获奖的年份

In [ ]:
# 假设 NOC 是你想要删除的列名
noc_names_to_remove = medal_counts['NOC'].unique()

# 删除对应的列
participation_by_year_country_without_medal = participation_by_year_country.drop(columns=noc_names_to_remove, errors='ignore')
participation_by_year_country_without_medal

In [ ]:
# 创建一个空列表来保存列名和第一次非0的行名
first_nonzero_rows = []

# 遍历每列，找到第一次非0的行
for col in medals_by_year.columns:
    # 获取每列第一次非0的行索引
    first_nonzero_index = medals_by_year[medals_by_year[col] != 0].index[0] if (medals_by_year[col] != 0).any() else None
    # 将列名和对应的第一次非0行索引添加到列表中
    first_nonzero_rows.append([col, first_nonzero_index])

# 将结果转换为 DataFrame
first_medal = pd.DataFrame(first_nonzero_rows, columns=['NOC', 'First Medal Year'])

# 去掉 'First Medal Year' 列小于 1920 的行
first_medal_filtered = first_medal[first_medal['First Medal Year'] >= 1920]

# 查看结果
first_medal_filtered


In [ ]:
# 创建一个空列表来保存列名和第一次非0的行名
first_nonzero_rows = []

# 遍历每列，找到第一次非0的行
for col in medals_by_year_gold.columns:
    # 获取每列第一次非0的行索引
    first_nonzero_index = medals_by_year_gold[medals_by_year_gold[col] != 0].index[0] if (medals_by_year_gold[col] != 0).any() else None
    # 将列名和对应的第一次非0行索引添加到列表中
    first_nonzero_rows.append([col, first_nonzero_index])

# 将结果转换为 DataFrame
first_medal = pd.DataFrame(first_nonzero_rows, columns=['NOC', 'First Medal Year'])

# 去掉 'First Medal Year' 列小于 1920 的行
first_medal_filtered = first_medal[first_medal['First Medal Year'] >= 1920]

# 查看结果
first_medal_filtered


## 2.3 构造数据集

In [ ]:
import pandas as pd
import numpy as np

# 假设 first_medal_filtered, participation_by_year_country, participation_by_year_country_count 和 sport_count_pivot 是现有的 DataFrame

# 新建一个空的列表来存储每行的数据
new_table_data = []

# 获取 first_medal_filtered 中的第一列内容（国家）和 First Medal Year
countries = first_medal_filtered['NOC'].values
first_medal_years = first_medal_filtered['First Medal Year'].values

# 遍历每个国家，填充已有的数据
for noc, first_medal_year in zip(countries, first_medal_years):
    if noc in participation_by_year_country.columns:  # 确保 noc 存在于 participation_by_year_country 的列中
        try:
            # 从 participation_by_year_country_count 获取该 NOC 和 Year 的数据
            participation_count = participation_by_year_country_count.loc[first_medal_year, noc] if first_medal_year in participation_by_year_country_count.index else 0

            # 从 participation_by_year_country 获取该 NOC 和 Year 的数据
            events_participation = participation_by_year_country.loc[first_medal_year, noc] if first_medal_year in participation_by_year_country.index else 0

            # 从 sport_count_pivot 获取该 NOC 和 Year 的数据
            sport_count = sport_count_pivot.loc[first_medal_year, noc] if first_medal_year in sport_count_pivot.index else 0

            # 将 NOC 和相关数据添加到新表格中
            new_table_data.append([noc, participation_count, events_participation, sport_count, 1])
        except KeyError:
            # 如果某个年份的数据不存在，则填充为 0
            new_table_data.append([noc, 0, 0, 0, 1])

# 第二次遍历，减去4年的数据
for noc, first_medal_year in zip(countries, first_medal_years):
    if noc in participation_by_year_country.columns:  # 确保 noc 存在于 participation_by_year_country 的列中
        first_medal_year_minus_4 = first_medal_year - 4  # 减去4年的年份
        try:
            # 从 participation_by_year_country_count 获取该 NOC 和 Year 的数据
            participation_count = participation_by_year_country_count.loc[first_medal_year_minus_4, noc] if first_medal_year_minus_4 in participation_by_year_country_count.index else 0

            # 从 participation_by_year_country 获取该 NOC 和 Year 的数据
            events_participation = participation_by_year_country.loc[first_medal_year_minus_4, noc] if first_medal_year_minus_4 in participation_by_year_country.index else 0

            # 从 sport_count_pivot 获取该 NOC 和 Year 的数据
            sport_count = sport_count_pivot.loc[first_medal_year_minus_4, noc] if first_medal_year_minus_4 in sport_count_pivot.index else 0

            # 将 NOC 和相关数据添加到新表格中
            new_table_data.append([noc, participation_count, events_participation, sport_count, 0])
        except KeyError:
            # 如果某个年份的数据不存在，则填充为 0
            new_table_data.append([noc, 0, 0, 0, 0])

# 第二次遍历，减去4年的数据
for noc, first_medal_year in zip(countries, first_medal_years):
    if noc in participation_by_year_country.columns:  # 确保 noc 存在于 participation_by_year_country 的列中
        first_medal_year_minus_4 = first_medal_year - 4  # 减去4年的年份
        try:
            # 从 participation_by_year_country_count 获取该 NOC 和 Year 的数据
            participation_count = participation_by_year_country_count.loc[first_medal_year_minus_4, noc] if first_medal_year_minus_4 in participation_by_year_country_count.index else 0

            # 从 participation_by_year_country 获取该 NOC 和 Year 的数据
            events_participation = participation_by_year_country.loc[first_medal_year_minus_4, noc] if first_medal_year_minus_4 in participation_by_year_country.index else 0

            # 从 sport_count_pivot 获取该 NOC 和 Year 的数据
            sport_count = sport_count_pivot.loc[first_medal_year_minus_4, noc] if first_medal_year_minus_4 in sport_count_pivot.index else 0

            # 将 NOC 和相关数据添加到新表格中
            new_table_data.append([noc, participation_count, events_participation, sport_count, 0])
        except KeyError:
            # 如果某个年份的数据不存在，则填充为 0
            new_table_data.append([noc, 0, 0, 0, 0])

# 获取所有唯一的年份
unique_years = athletes['Year'].unique()

# 遍历所有年份
for year in unique_years:
    # 筛选出该年且 Medal 为 'No medal' 的数据
    if year < 1996:
        continue
    no_medal_athletes = athletes[(athletes['Year'] == year) & (athletes['Medal'] == 'No medal')]

    # 获取所有 NOC 中 Medal 只有 'No medal' 的 NOC
    no_medal_nocs = athletes[(athletes['Year'] == year)].groupby('NOC').filter(lambda group: (group['Medal'] == 'No medal').all())['NOC'].unique()

    # 遍历每个 NOC
    for noc in no_medal_nocs:
        try:
            # 从 participation_by_year_country_count 获取该 NOC 和 Year 的数据
            participation_count = participation_by_year_country_count.loc[year, noc] if year in participation_by_year_country_count.index and noc in participation_by_year_country_count.columns else 0

            # 从 participation_by_year_country 获取该 NOC 和 Year 的数据
            events_participation = participation_by_year_country.loc[year, noc] if year in participation_by_year_country.index and noc in participation_by_year_country.columns else 0

            # 从 sport_count_pivot 获取该 NOC 和 Year 的数据
            sport_count = sport_count_pivot.loc[year, noc] if year in sport_count_pivot.index and noc in sport_count_pivot.columns else 0

            # 将 NOC 和相关数据添加到新表格中
            new_table_data.append([noc, participation_count, events_participation, sport_count])

        except KeyError:
            # 如果某个年份或 NOC 数据不存在，则填充为 0
            new_table_data.append([noc, year, 0, 0, 0])

# 创建新表格
tree_dataset = pd.DataFrame(new_table_data, columns=['NOC', 'Participation_count', 'Events_Participation', 'Sport_Count', 'Will_Earn_Medal'])

# 填充缺失值为0
tree_dataset = tree_dataset.fillna(0)

# 删除全零行
tree_dataset = tree_dataset.loc[~(tree_dataset.iloc[:, 1:-2].eq(0)).all(axis=1)]

# 显示最终的表格
tree_dataset

## 2.4 随机森林

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

# 假设 tree_dataset 是我们之前创建的 DataFrame
# tree_dataset = ...

# 1. 准备数据：将特征和目标变量分开
X = tree_dataset[['Participation_count', 'Events_Participation', 'Sport_Count']]  # 特征
y = tree_dataset['Will_Earn_Medal']  # 目标变量

# 2. 数据预处理（可选）：例如，标准化特征值
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 3. 拆分数据集：将数据分为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 4. 定义随机森林模型
rf_model = RandomForestClassifier(random_state=42, class_weight='balanced')

# 5. 定义超参数范围
param_grid = {
    'n_estimators': [50, 100, 150, 200],  # 树的数量
    'max_depth': [None, 10, 20, 30],  # 树的最大深度
    'min_samples_split': [2, 5, 10],  # 拆分节点的最小样本数
    'min_samples_leaf': [1, 2, 4],  # 叶节点的最小样本数
    'max_features': ['auto', 'sqrt', 'log2'],  # 每个树考虑的最大特征数
}

# 6. 使用 GridSearchCV 进行超参数搜索
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2, scoring='accuracy')
grid_search.fit(X_train, y_train)

# 7. 输出最佳超参数组合
print(f"Best Hyperparameters: {grid_search.best_params_}")

# 8. 使用最佳模型进行预测
best_rf_model = grid_search.best_estimator_

# 9. 在测试集上进行预测
y_pred = best_rf_model.predict(X_test)
y_pred_prob = best_rf_model.predict_proba(X_test)[:, 1]  # 获取“Will_Earn_Medal=1”的概率

# 10. 计算模型性能（准确率和AUC）
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_prob)

print(f'Accuracy: {accuracy:.4f}')
print(f'ROC AUC: {roc_auc:.4f}')

# 11. 绘制混淆矩阵
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['No Medal', 'Medal'], yticklabels=['No Medal', 'Medal'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

# 12. 绘制 ROC 曲线
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()


In [ ]:
# 7. 预测新数据的 "Will_Earn_Medal" 概率
# 假设我们有一组新的输入数据
new_data = pd.DataFrame({
    'Participation_count': [5, 10, 3],  # 新输入数据
    'Events_Participation': [8, 15, 4],
    'Sport_Count': [3, 7, 2]
})

# 对新数据进行标准化处理
new_data_scaled = scaler.transform(new_data)

# 预测新数据的概率
predicted_probabilities = best_rf_model.predict_proba(new_data_scaled)[:, 1]  # 获取 "Will_Earn_Medal" 为1的概率

# 打印新数据的预测概率
print('Predicted Probabilities for Will_Earn_Medal (1):')
print(predicted_probabilities)

## 2.4 构造测试集

In [ ]:
# 筛选出 Medal 列为 'No medal' 的数据
no_medal_athletes = athletes[athletes['Medal'] == 'No medal']

# 获取所有 NOC 中 Medal 只有 'No medal' 的 NOC
no_medal_nocs = athletes.groupby('NOC').filter(lambda group: (group['Medal'] == 'No medal').all())['NOC'].unique()

# 打印结果
no_medal_nocs

In [ ]:
# 新建一个空的列表来存储每行的数据
new_table_data = []

# 遍历没有获得奖牌的国家（no_medal_nocs）
for noc in no_medal_nocs:
    first_medal_year = 2024  # 固定 first_medal_year 为 2024
    try:
        # 从 participation_by_year_country_count 获取该 NOC 和 Year 的数据
        participation_count = participation_by_year_country_count.loc[first_medal_year, noc] if first_medal_year in participation_by_year_country_count.index and noc in participation_by_year_country_count.columns else 0

        # 从 participation_by_year_country 获取该 NOC 和 Year 的数据
        events_participation = participation_by_year_country.loc[first_medal_year, noc] if first_medal_year in participation_by_year_country.index and noc in participation_by_year_country.columns else 0

        # 从 sport_count_pivot 获取该 NOC 和 Year 的数据
        sport_count = sport_count_pivot.loc[first_medal_year, noc] if first_medal_year in sport_count_pivot.index and noc in sport_count_pivot.columns else 0

        # 将 NOC 和相关数据添加到新表格中
        new_table_data.append([noc, participation_count, events_participation, sport_count])

    except KeyError:
        # 如果某个年份或 NOC 数据不存在，则填充为 0
        new_table_data.append([noc, 0, 0, 0])

# 创建新表格
test_dataset = pd.DataFrame(new_table_data, columns=['NOC', 'Participation_count', 'Events_Participation', 'Sport_Count'])

# 填充缺失值为0
test_dataset = test_dataset.fillna(0)

# 删除全零行
test_dataset = test_dataset.loc[~(test_dataset.iloc[:, 1:].eq(0)).all(axis=1)]

# 显示最终的表格
test_dataset


## 2.5 测试

In [ ]:
# Step 1: 存储每列的预测概率
predicted_probabilities_all_columns = []

# Step 2: 遍历每一行（每个国家），对列中的特征进行预测
for _, row in test_dataset.iterrows():
    # 提取当前行的特征（即每个国家的 Participation_count, Events_Participation, Sport_Count）
    features = row[['Participation_count', 'Events_Participation', 'Sport_Count']].values.reshape(1, -1)  # Reshaping to (1, 3)
    print(features)

    # 预测概率（假设rf_model已经训练好）
    probabilities = best_rf_model.predict_proba(features)

    # 获取预测为1（获得奖牌）的概率
    predicted_probabilities = probabilities[:, 1]

    # 将预测概率存储到列表中
    predicted_probabilities_all_columns.append(predicted_probabilities[0])  # 取出每行的预测概率

print("Predicted Probabilities (before ranking):", predicted_probabilities_all_columns)

# Step 3: 对预测概率进行排名
# 获取排序后的索引
ranked_probabilities = np.argsort(predicted_probabilities_all_columns)

# Step 4: 绘制柱状图
plt.figure(figsize=(10, 6))
plt.bar(test_dataset['NOC'], predicted_probabilities_all_columns)
plt.xlabel('Country')
plt.ylabel('Predicted Probability')
plt.title('Predicted Probabilities for Medal')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()


# 3 相关性

In [ ]:
# 对除前四列外的所有列进行处理
for col in programs.columns[4:]:
    # 用正则表达式去除非数字字符
    programs[col] = programs[col].replace(r'\D', '', regex=True)

    # 将空缺值补零
    programs[col] = programs[col].fillna('0')

# 删除索引为 0, 1, 3 的列
done_programs = programs.drop(programs.columns[[0, 1, 3, 7]], axis=1)

# 将原 'Code' 列作为新的列名
done_programs = done_programs.set_index('Code')

# 转置 DataFrame
done_programs_transposed = done_programs.transpose()

# 删除全0的列
done_programs_transposed = done_programs_transposed.loc[:, (done_programs_transposed != '0').any(axis=0)]

# 输出处理后的 DataFrame
done_programs_transposed

In [ ]:
medals_by_year

## 3.1 皮尔逊相关系数矩阵

In [ ]:
# 假设 medals_by_year 和 done_programs_transposed 是你的 DataFrame

# 先统一将 done_programs_transposed 和 medals_by_year 的索引对齐
done_programs_transposed = done_programs_transposed.reset_index(drop=True)

# 遍历每个国家（NOC）列
for NOC in hosts['NOC']:
    if pd.isna(NOC):
        continue
    # 确保 'NOC' 列是数值型，转换时忽略无法转换的值（设置为NaN）
    choice_column = pd.to_numeric(medals_by_year[NOC], errors='coerce').reset_index(drop=True)

    # 将 'NOC' 列与 done_programs_transposed 合并
    df_combined = pd.concat([choice_column, done_programs_transposed], axis=1)

    # 去除含有 NaN 的行
    df_combined = df_combined.dropna()

    # 计算皮尔逊相关系数矩阵
    correlation_matrix = df_combined.corr(method='pearson')

    # 获取当前国家列的相关性
    gre_corr = correlation_matrix[NOC].drop(NOC)  # 排除当前国家本身的相关性

    # 获取相关性最大的前3项
    top_corr = gre_corr.nlargest(3)  # 获取最大相关性前3项

    # 输出相关性最大的前3项
    print(f"Top 3 highest correlations with {NOC}:")
    for col, corr_value in top_corr.items():
        print(f"{col}: {corr_value:.2f}")

    # 绘制每个国家的相关性热力图
    # sns.set(font_scale=0.8)
    # plt.figure(figsize=(10, 8))
    # sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
    # plt.title(f'Pearson Correlation Matrix for {NOC}', fontsize=16)
    # plt.show()

In [ ]:
import pandas as pd

# 假设 medals_by_year 和 done_programs_transposed 是你的 DataFrame

# 先统一将 done_programs_transposed 和 medals_by_year 的索引对齐
done_programs_transposed = done_programs_transposed.reset_index(drop=True)

# 用于存储每个 NOC 的相关性
correlation_dict = {}

# 遍历每个国家（NOC）列
for NOC in medals_by_year.columns:
    if pd.isna(NOC) or NOC == 'POL':
        continue
    
    # 确保 'NOC' 列是数值型，转换时忽略无法转换的值（设置为NaN）
    choice_column = pd.to_numeric(medals_by_year[NOC], errors='coerce').reset_index(drop=True)

    # 将 'NOC' 列与 done_programs_transposed 合并
    df_combined = pd.concat([choice_column, done_programs_transposed], axis=1)

    # 去除含有 NaN 的行
    df_combined = df_combined.dropna()

    # 计算皮尔逊相关系数矩阵
    correlation_matrix = df_combined.corr(method='pearson')

    # 将当前国家与所有列的相关性存入字典
    correlation_dict[NOC] = correlation_matrix[NOC].drop(NOC)  # 排除自身的相关性

# 将字典转换为 DataFrame 并转置
correlation_df = pd.DataFrame(correlation_dict).transpose()

# 去除含有 NaN 的行
correlation_df = correlation_df.dropna()

# for index, row in correlation_df.iterrows():
#     # Get the indices of the top 3 largest values in the row
#     to_indices = row.nlargest(3).index
#     # Set all values in the row to 0
#     correlation_df.loc[index] = 0
#     # Set the top 3 largest values to 1
#     correlation_df.loc[index, to_indices] = 1

# 去除最后三列
correlation_df = correlation_df.drop(correlation_df.columns[-3:], axis=1)

correlation_df


In [ ]:
# 存储每行最大的三个值和列名
max_values = []
for row_index, row in correlation_df.iterrows():
    # 获取当前行最大的三个元素及其列名
    top_3 = row.nlargest(3)
    # 创建一个字典包含列名和值
    row_max_values = {'Row Name': row_index, 'Top 3 Columns': list(top_3.index), 'Top 3 Values': list(top_3.values)}
    max_values.append(row_max_values)

# 将这些数据转换为新的 DataFrame
max_df = pd.DataFrame(max_values)

# 打印新表
max_df

## 3.2 最小二乘数据生成

### 数据集生成

In [ ]:
# 删除含有NaN的行
hosts_cleaned = hosts.dropna()

# 删除第一列
hosts_cleaned = hosts_cleaned.drop(hosts_cleaned.columns[1], axis=1)

hosts_cleaned

In [ ]:
for i in range(4):
    # Generate years from 1948 to 2020 with a step of 4
    years = np.arange(1948, 2021, 4)
    add_lst = hosts_cleaned.copy()
    # Add random years to the 'Year' column in hosts_cleaned
    add_lst['Year'] = np.random.choice(years, size=len(add_lst))
    
    # Fill the 'Year' column with random values from medals_by_year.columns
    random_columns = np.random.choice(medals_by_year.columns, size=len(add_lst))
    
    # Output the modified DataFrame
    add_lst['NOC'] = random_columns  # Replace Year with random column names
    
    # Vertically concatenate hosts_cleaned and add_lst
    hosts_cleaned = pd.concat([hosts_cleaned, add_lst], ignore_index=True)

# Output the modified DataFrame
hosts_cleaned

In [ ]:
sport_count_year

In [ ]:
# 假设 sport_count_year 是包含 'NOC', 'Year' 和 'Sport_Count' 列的 DataFrame

# 合并 hosts_cleaned 和 medal_counts 表，按 'NOC' 和 'Year' 列进行连接
merged_df = pd.merge(hosts_cleaned, medal_counts[['NOC', 'Year', 'Total']], on=['NOC', 'Year'], how='left')

# 添加一列名为 'host'，值全为 1
merged_df = merged_df.assign(host=1)

# 进一步合并 sport_count_year 表，按 'NOC' 和 'Year' 列进行连接
merged_df = pd.merge(merged_df, sport_count_year[['NOC', 'Year', 'Sport']], on=['NOC', 'Year'], how='left')

final_df = pd.merge(merged_df, correlation_df, left_on='NOC', right_index=True, how='inner')

final_df = final_df.dropna()

# Set the first 27 rows of the 'host' column to 1, and the rest to 0
final_df['host'] = 0  # Set all rows to 0
final_df.loc[:26, 'host'] = 1  # Set the first 27 rows (index 0 to 26) to 1

# Output the modified final_df
final_df

In [ ]:
import statsmodels.api as sm

# 假设 'final_df' 是已经合并好的 DataFrame，并且我们希望对 'Total' 进行回归

# 选择所有列作为自变量
X = final_df.drop(columns=['Total','Year','NOC'])  # 删除因变量 'Total' 列，所有其他列作为自变量

# 因变量
Y = final_df['Total']

# 在自变量中添加常数项（截距项）
X = sm.add_constant(X)

# 构建并拟合回归模型
model = sm.OLS(Y, X)  # OLS：普通最小二乘回归
results = model.fit()

# 输出回归结果的总结
results.summary()
